# Hello, `vizarr`

This example explains the basic usage of `vizarr` as an Imjoy Plugin in a jupyter notebook. 

### Create a multiscale OME-Zarr

The easiest way to use `vizarr` in a jupyter notebook is to have an OME-Zarr compliant image. OME-Zarr is a developing standard from the OME-Community and currently considered experimental as it is under active development. The specification can be found [here](https://github.com/ome/omero-ms-zarr/blob/master/spec.md). 
 
Creating an OME-Zarr for your own images can be accomplished by using Bio-Formats, `bioformats2raw` with the `--file_type=zarr --dimension-order=XYZCT` options and adding `omero` metadata to the root attrs of the resulting multiscale group. 

For convenience, we have included a simple function to create a multiscale OME-Zarr below.

In [ ]:
from create_fixture import create_ome_zarr
create_ome_zarr("astronaut.zarr") # creates an example OME-Zarr in the current directory

### Open OME-Zarr in jupyter with `zarr-python`

The root of an OME-Zarr is a group that aheres to the `multiscales` zarr extension (if pyramidal) and also contains `omero` metadata describing _how_ to render the image. Here we open the OME-Zarr as a `zarr.Group` using `zarr-python`. 

In [ ]:
import zarr
multiscale_astronaut = zarr.open("astronaut.zarr", mode="r") # open the zarr created above in jupyter kernel

### Create `vizarr` image and view via Imjoy Plugin

A `vizarr` image is simply a python dictionary specifying how to initially render the zarr-based store in the viewer. The `source` field must be a `zarr.Array` or `zarr.Group`. If the `zarr.Group` is for an OME-Zarr, `source` is the only required field. If it is an `zarr.Array` or `zarr.Group` that is not OME-Zarr, additional fields are required for rendering (described below).

In [ ]:
from imjoy_plugin import run_vizarr

# Create Zarr 
astronaut_img = { "source": multiscale_astronaut }

# Run vizarr
run_vizarr(astronaut_img)

### Create a generic array layer

`vizarr` also supports custom `zarr.Array` and `zarr.Group`s, but requires additional metadata to render. Viewing a custom `zarr` has the following requirements:

- The last two dimensions of the `zarr.Array` must be `YX`.
- If an `zarr.Array` has more than 2D dimensions, non-YX axis much have a `chunksize` of `1`.
- If a `zarr.Group` is provided, it must implement the `multiscales` specification and subresolution arrays must adhere to the same `zarr.Array` requirements outlined above.

In [ ]:
import numpy as np

# Create in-memory numpy array
arr = np.random.randint(0, 255, (1, 1024, 1024), dtype=np.uint8)

# Wrap array as `zarr.Array`
z_zarr = zarr.array(arr)

# Create a vizarr image from custom zarr.Array
noise = {
    "source": zarr.array(arr), 
    "dimensions": "cyx", # if not OME-Zarr, need to label dimensions for array
    # Manually define list of 2D panes to additively blend
    "channels": [{ 
        "selection": { "c": 0 },     # required, defines indexing along non-XY axes to get 2D pane
        "contrast_limits": [0, 255], # optional (but recommended), defaults to min/max of dtype
        "color": "#FFFFFF",          # optional, default is #FFFFFF
        "label": "channel_0",        # optional, default channel_x
    }],
    "opacity": 0.5,   # optional, default 1
    "name": "noise",  # optional, default image_x
}

### View multiple images

`run_vizarr` takes a list of images which can be viewed in the same interactive scene as different image layers.

In [ ]:
run_vizarr([astronaut_img, noise])

### Set initial view state

The initial view state can be defined as a second argument for `run_vizarr` as well, but is optional.

In [ ]:
run_vizarr([astronaut_img, noise], view_state={ "zoom": -3, "target": [512, 512, 0] })